<a href="https://colab.research.google.com/github/alisony755/DS4400/blob/main/HW2/DS4400_HW2_Problem3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem 3

In [ ]:
# Import numpy and pandas
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Import Google Drive to upload data
from google.colab import drive
drive.mount('/content/drive')

# Read in data from file path
file_id = '1_hNHlID38Uf5lphkEyXVKz5_afFb1eve'
file_path = f"https://drive.google.com/uc?id={file_id}&export=download"
df_house = pd.read_csv(file_path)

# Drop unnecessary columns
df_house = df_house.drop(columns=['id', 'date', 'zipcode'])

# Print first few rows of data
df_house.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,47.5112,-122.257,1340,5650
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,47.7210,-122.319,1690,7639
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,47.7379,-122.233,2720,8062
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,47.5208,-122.393,1360,5000
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,47.6168,-122.045,1800,7503


In [ ]:
# Separate features and response
X = df_house.drop(columns=['price'])
y = df_house['price'] / 1000 # Divide price by 1000 to reduce value of the MSE

### 3.1

In [ ]:
# Test-train split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
def add_bias(X):
  """ Adds a bias term (column of 1s) to the given feature maxtrix for the intercept

  Args:
    X (numpy.ndarray): Feature matrix

  Returns:
    numpy.ndarray: Feature matrix with bias column

  """

  ones = np.ones((X.shape[0], 1))
  return np.hstack((ones, X))

In [ ]:
def closed_form_train(X, y):
  """ Trains a multiple linear regression model using the closed-form normal equation

  Args:
    X (numpy.ndarray): Feature matrix of shape (n_samples, n_features)
    y (numpy.ndarray): Response vector of shape (n_samples,) or (n_samples, 1)

    Returns:
        numpy.ndarray: Parameter vector theta of shape (n_features + 1,),
                       where theta[0] is the intercept and the remaining entries are coefficients

  """

  # Add bias term
  X_b = add_bias(X)

  # Compute theta using the normal equation
  XtX = X_b.T @ X_b
  XtX_inv = np.linalg.inv(XtX)
  XtY = X_b.T @ y

  theta = XtX_inv @ XtY
  return theta

In [ ]:
def predict(X, theta):
    """ Predicts the response variable for new data using the trained linear regression parameters

    Args:
        X (numpy.ndarray): Feature matrix of shape (n_samples, n_features).
        theta (numpy.ndarray): Parameter vector of shape (n_features + 1,),
                               where theta[0] is the intercept

    Returns:
        numpy.ndarray: Predicted response vector of shape (n_samples,)

    """

    X_b = add_bias(X)
    return X_b @ theta

In [ ]:
# Train model using closed-form solution
theta = closed_form_train(X_train_scaled, y_train.to_numpy())

In [ ]:
# Make predictions for training data
y_train_pred = predict(X_train_scaled, theta)
print(f"Training predictictions: {y_train_pred}\n")

# Make predictions for testing data
y_test_pred = predict(X_test_scaled, theta)
print(f"Testing predictictions: {y_test_pred}")

Training predictictions: [493.08621136 232.51519657 126.94706397 ... 480.11256975 -59.31691301
 475.67457038]

Testing predictictions: [ 460.6395007   770.75259853 1217.74164694 ...  396.24002165  572.5106148
  415.16475948]


### 3.2

In [ ]:
# Compute and print MSE and R^2 for training data
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

print(f"Training MSE: {train_mse}")
print(f"Training R^2: {train_r2}\n")

# Compute and print MSE and R^2 for testing data
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"Testing MSE: {test_mse}")
print(f"Testing R^2: {test_r2}")

Training MSE: 40566.14884268924
Training R^2: 0.6895018808268092

Testing MSE: 46741.73364670792
Testing R^2: 0.6908139292823656
